**извлечение признаков из текста на естественном языке: word2vec**

Евгений Борисов borisov.e@solarl.ru

In [53]:
import sys
import re
import gzip
import numpy as np
# import scipy.io

import matplotlib.pyplot as plt

In [21]:
# загружаем предварительно очищенный текст
# разрезаем текст на слова
# удаляем пустые элементы
with gzip.open('../data/text/text.txt.gz','rt') as f: 
    text = [ w for w in f.read().split() if w ]
    
# text    

In [25]:
words = sorted(set(text)) # словарь из текста
# words

In [ ]:
# # сохраняем словарь
# with open("result/words.txt", "w") as f:
#     for w in words:
#         f.write("%s\n"%w)

In [26]:
# заменяем слова в тексте их номерами в словаре
vocab =  { words[i]:i for i in range(0,len(words)) }
code = [ vocab[w] for w in text ]

In [29]:
n = len(words) # количество слов в словаре
m = len(code) # количество слов в тексте 
print( "размер словаря: %i слов" % n )
print( "размер текста: %i слов" % m )

размер словаря: 355 слов
размер текста: 556 слов


In [30]:
# унитарное кодирование словаря (one-hot-encoding)
Ve = np.eye( n ) 

# кодированный текст (последовательность кодов слов)
Te = Ve[code,:] 

In [40]:
# Te

In [50]:
# формируем учебный набор...

c = 3  # размер окна контекста, количество слов перед текущим словом и после него
W = [] # слова
C = [] # контекст для слов

for i in range(c,m-c) :
    W.append( Te[i,:] ) # код текущего слова
    # контекст для текущего слова
    C.append( np.vstack( ( Te[i-c:i,:] , Te[i+1:i+c+1,:] ) ).reshape(1,2*c,n) )

In [51]:
W = np.vstack(W)
W.shape

(550, 355)

In [52]:
C = np.concatenate(C)
C.shape

(550, 6, 355)

In [ ]:
# np.savez( "result/data.npz", W=W, C=C )
# scipy.io.savemat("result/data.mat", dict(W=W, C=C))

---

In [ ]:
# ф-ция активации скрытого слоя - линейная
# def act(s): return s

# ф-ция активации выходного слоя
def softmax(s): 
    e = np.exp(s)
    return e/e.sum(axis=1).reshape(s.shape[0],1)

In [ ]:
def w2v_step(W,Vi,Vo):
    H = W.dot(Vi) # значения скрытого слоя
    U = H.dot(Vo) # состояния выходного слоя
    O = softmax(U) # выход сети
    return H,U,O


Функция потери на учебном примере $i$

$$E_i = \left| \log\left( \sum\limits_i \exp(U_i) \right) - \sum\limits_i\sum\limits_j (U_i * Q_{ij}) \right| $$


$U_{i}$ состояние выходного слоя для слова $i$     
$Q_{ij}$ слово $j$ контекста слова $i$   
где ∗ - операция поэлементного умножения векторов

In [ ]:
# function [v,w] = w2v_weigth_init(n,k),
#    % n = 10 ; % количество слов в словаре
#    % k = 30 ; % размер скрытого слоя H
#    v = normrnd( zeros(n,k),ones(n,k)*1e-2 ) ; % веса связей от входного к скрытому слою
#    w = normrnd( zeros(k,n),ones(k,n)*1e-2 ) ; % веса связей от скрытого к выходному слою
# endfunction


In [ ]:
# function [vn,wn] = w2v_weigth_norm(v,w) ;
#    n = norm( [ v(:), w(:) ] ) ;
#    vn = v./n;
#    wn = w./n;
# endfunction

In [ ]:
# function [vn,wn] = w2v_weigth_add(v,w,dv,dw,a),
#    vn = v + dv.*a ;
#    wn = w + dw.*a ; 
# endfunction

In [ ]:
def w2v_loss(U,C):
    n,v,cws = C.shape 
        # количество примеров
        # количество слов в словаре
        # размер окна контекста   
    
    Us = np.log( np.exp(U).sum(axis=1) ).sum(axis=0) ;
    
    Uo = 0.0;
    for i in range(cws): # для всех слов контекста
        Ci = C[:,i,:].reshape([n,v]) # набор слов контекста i
        Uo += (U*Ci).sum() # значения выходного слоя для слов x контекста i
    
    return np.abs(Us-Uo)/n

In [ ]:
def w2v_grad(C,W,H,O,Vo):
    s = C.shape # размеры набора слов контекста
    cws = s[1] # размер окна контекста 
    # sw = Vo.shape # размеры матрицы связей выходного слоя

    gVi = gVo = 0.0 
    
    for i in range(cws):
        Ci = C[:,i,:].reshape( s([1,3]) ) # слово i контекста
        D = O - Ci # ошибка на слове контекста i
        gVo += H.T.dot(D)
        gVi += D.T.dot(W).dot(Vo.T)  
    
    return gVi,gVo

In [ ]:
# w2v_fit_skip_gram
#  вычисляем состояния слоёв и выход
#  вычисляем ошибку
#  вычисляем градиент ф-ции потери
#  нормируем значения градиента
#  корректируем веса




# % - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
# function [Vi0,Vo0,er] = w2v_fit_skip_gram(C,W,Vi,Vo),
#    a = 0.91; % скорость обучения

#    nn = 1000 ;
#    er = [ 1e10 ] ; % история изменения ошибки
#    for i=1:nn,
#       [H,U,O] = w2v_step(W,Vi,Vo) ; % вычисляем состояния слоёв и выход

#       er(end+1) = w2v_loss(U,C) ; % вычисляем ошибку 
#       printf("[i] эпоха %i/%i, ошибка обучения: %f     \r",i,nn, er(end)) ;

#       if( er(end-1) < er(end) ) break; endif; % если ошибка растёт то останавливаем обучение 

#       [gVi,gVo] = w2v_grad(C,W,H,O,Vo) ; % вычисляем градиент ф-ции потери

#       [gVi,gVo] = w2v_weigth_norm(gVi,gVo) ; % нормируем значения градиента

#       Vi0 = Vi ; 
#       Vo0 = Vo ; 

#       [Vi,Vo] = w2v_weigth_add(Vi,Vo,gVi,gVo,-a) ; % корректируем веса
#    endfor

#    er(1) = [] ;
#    er(end) = [] ;

#    printf("[i] эпоха %i/%i, ошибка обучения: %f     \n",i,nn, er(end)) ;

# endfunction

In [ ]:
# # выход сети
# def run(x,Vi,Vo): return softmax( x.dot(Vi).dot(Vo) )


# # прямой проход
# def forward(x,Vi,Vo):
#     L = [ x.dot(Vi) ] # состояние (не активированное) скрытого слоя
#     L.append( L[0].dot(Vo) ) # состояние (не активированное) выходного слоя
#     return L


# # обратный проход
# def backward(L,y):
#     O = act(L[1]) # выход сети
#     E = [ (O-y)*act_drv(L[1]) ] # ошибка выходного слоя
#     E.insert(0, E[0].dot(W1.T)*act_drv(L[0]) ) # ошибка скрытого слоя
#     return E


# # градиент
# def grad(L,E):
#     GW = [ X.T.dot(E[0]) ] # градиент по весам скрытого слоя
#     GS = [ E[0].sum(axis=0) ] # градиент по сдвигам скрытого слоя

#     O = act(L[0]) # выход скрытого слоя
#     GW.append( O.T.dot(E[1]) ) # градиент по весам выходного слоя
#     GS.append( E[1].sum(axis=0) ) # градиент по сдвигам выходного слоя

#     return GW,GS


# # нормируем градиент
# def grad_norm(gw,gs):
#     mw = np.abs(np.hstack([ gw[0].flatten(), gw[1].flatten(), gs[0], gs[1], ])).max()
        
#     if mw != 0.0:
#         gw[0] /= mw
#         gw[1] /= mw
#         gs[0] /= mw
#         gs[1] /= mw
    
#     return gw,gs

In [ ]:
# # инициализация весов и сдвигов
# W0 = np.random.normal(loc=0.0, scale=0.1, size=[X.shape[1],s_layer])
# S0 = np.zeros(s_layer)

# W1 = np.random.normal(loc=0.0, scale=0.1, size=[s_layer,y.shape[1] ])
# S1 = np.zeros(y.shape[1])

# # метод градиентного спуска

# a=0.05 # скорость обучения
# r=0.001 # регуляризация
# m=0.001 # момент

# # ex_count = X.shape[0]   # количество примеров

# # максимальное число циклов обучения
# MAX_ITER = 800

# MIN_ERROR = 0.1 # порог минимальной ошибки

# err =[1e7] 

# dW1=dW0=dS0=dS1=0.0

# for i in range(MAX_ITER):
#     O = run(X) # выход сети
#     err.append( loss(O,y) ) # история значений ф-ции потери
    
#     if err[-1] < MIN_ERROR: # проверяем достижение порога
#         break

#     L=forward(X) # прямой проход
#     E=backward(L,y) # обратный проход
#     GW,GS = grad(L,E) # градиент
#     GW,GS = grad_norm(GW,GS) # нормируем градиент
    
    
#     dW0 = a*( GW[0]+ r*W0 ) + m*dW0
#     dW1 = a*( GW[1]+ r*W1 ) + m*dW1
#     dS0 = a*GS[0]+ m*dS0
#     dS1 = a*GS[1]+ m*dS1
    
#     # изменяем веса и сдвиги
#     W0 -= dW0
#     W1 -= dW1 
#     S0 -= dS0
#     S1 -= dS1

# print('step:',i,'/',MAX_ITER)
# print('error:',err[-1],'/',MIN_ERROR)

# # изменение ошибки обучения
# fig, ax = plt.subplots()
# ax.plot(err[2:])
# plt.grid()
# plt.show()